In [24]:
## for jupyter
import logging
logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
logger.setLevel(logging.INFO)

fhandler = logging.FileHandler(filename='moneyWorks4me.txt', mode='w')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)

# logger.
# logging.info('This is a warning message')

In [25]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests 
from urllib.request import urlopen, Request
from urllib.parse import urljoin
import time
import re
import json

In [4]:
root_url = "https://www.moneyworks4me.com/"

In [5]:
moneyWorksForMeCode = {"Reliance":"100325"}

In [37]:
class moneyWorksForMe():
    
    def __init__(self,comp_code,to_page=2):
        self.comp_code = comp_code
        self.to_page = to_page
        
        self.url = "https://www.moneyworks4me.com/stock-markets/search/news/fincode/" + comp_code #+ "/page/" + from_page
    
    def get_html(self,url):
        agent = "Mozilla/5.0 (platform; rv:geckoversion) Gecko/geckotrail Firefox/firefoxversion"
        return urlopen(Request(url,headers = {'User-Agent': agent})).read()
    
    def get_news_content(self,url):
        
        time.sleep(3)
        s = bs(
                self.get_html(
                url), "html.parser")
        
        div = s.find("div",{"class":"col-12 sub-heading-3 text-darkgray"})
        
        content = str(div.text)
        content = content.strip()
        
        return content
    
    def crawl(self):

        scraped_data = {'link' : [], 'title' : [], 'date' : [], 'news_text' : [] }
        if self.to_page > 0:
            for i in range(0,self.to_page):
                time.sleep(3)
                if i == 0:
                            # first url
                    url = self.url
                else:
                    # url/page/2 # no page 1 there in URL
                    url = self.url + "/page/"+str(i+1)
                
                start_time = time.time()
                logging.info(f"fetching url:{url}")
                    
                s = bs(
                self.get_html(
                url), "html.parser")
                
                divs = s.find_all("div",{"class":"col-md-12 gray-card-light newswise-conatiner mt-3"})
                
                for div in divs:
                    inner_div = div.find("div",{"class":"d-flex sm-d-block"})

                    date_div = inner_div.find("div",{"class":
                                                    "newswise-date text-center w-25 align-self-center col-md-2 justify-content-center sm-w-100 sm-d-flex"})

                    title_div = inner_div.find("div",
                                              {"class":"news-info-section col-md-10 align-self-center"})

                    date = date_div.text.strip().split("\n")

                    scraped_data['date'].append(" ".join(date))

                    scraped_data['title'].append(title_div.a.text.strip())

                    scraped_data['link'].append(urljoin(root_url,title_div.a['href']))

                    scraped_data['news_text'].append(self.get_news_content(
                    urljoin(root_url,title_div.a['href'])
                    ))
                
                end_time = time.time()
                logging.info(f"time taken : {round((end_time-start_time),2)} seconds")
            
            return scraped_data

        
#     def scrape(self):

In [38]:
reliance = moneyWorksForMe(moneyWorksForMeCode['Reliance'],to_page=110)

In [39]:
%%time
df = pd.DataFrame(reliance.crawl())

Wall time: 1h 13min 9s


In [40]:
df.head()

,link,title,date,news_text
0,https://www.moneyworks4me.com/company/news/ind...,Reliance Industries’ step down arm inks agreem...,"05 Aug, 2022",Reliance Industries' step down arm -- Reliance...
1,https://www.moneyworks4me.com/company/news/ind...,Reliance Industries secures 104th position in ...,"04 Aug, 2022",Reliance Industries (RIL) has jumped 51 places...
2,https://www.moneyworks4me.com/company/news/ind...,Reliance Industries gains as its arm gets CCI ...,"03 Aug, 2022",Reliance Industries is currently trading at Rs...
3,https://www.moneyworks4me.com/company/news/ind...,CCI approves stake buy in SCIPL by Reliance In...,"03 Aug, 2022",The Competition Commission of India (CCI) has ...
4,https://www.moneyworks4me.com/company/news/ind...,Reliance Industries rises as its arm acquires ...,"02 Aug, 2022",Reliance Industries is currently trading at Rs...


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   link       1100 non-null   object
 1   title      1100 non-null   object
 2   date       1100 non-null   object
 3   news_text  1100 non-null   object
dtypes: object(4)
memory usage: 34.5+ KB


In [44]:
df.to_csv(".\\..\\data\\moneyWorks4me_Reliance_history.csv",encoding='utf-8-sig',index=False)